
# Unity Catalog: Control de Accesos y Seguridad


## Bloque 1: Intro Unity Catalogo

### 1.1 Rol de Unity Catalog
En Unity Catalog normalmente se sigue **este patrón**:

**Un único metastore** → nivel más alto.

Dentro del metastore → **varios catálogos que segmentan grandes ámbitos**:
  - Por dominio de negocio: finance, sales, hr…
  - O por entorno: dev, test, prod.

Dentro de cada catálogo → **esquemas que organizan áreas más concretas** (ej. dentro de sales: bronze, silver, gold o crm, orders, etc.).

**Dentro de los esquemas → tablas, vistas, funciones, modelos.**

👉 Lo que resuelve UC es tener un metastore centralizado que te permita:

  - Ver todos los catálogos y esquemas disponibles en un solo sitio.
  - Aplicar seguridad y lineage de forma coherente.
  - Evitar que cada departamento levante “su Hive metastore paralelo” que nadie más puede ver.

Aporta: metastore unificado, permisos centralizados, lineage automático, descubrimiento de datos, data sharing seguro.   


### 1.2 Jerarquía y definiciones
- **Metastore**: la “biblioteca” central de metadatos. Describe **qué** objetos existen (tablas, vistas, funciones, modelos), **dónde** están y **con qué propiedades**.  
- **Catálogo**: contenedor de **esquemas**. Suele separar dominios (p. ej., `finance`, `hr`) o **entornos** (`dev`, `test`, `prod`).  
- **Esquema**: contenedor dentro del catálogo. Suele mapear capas funcionales (p. ej., `bronze`, `silver`, `gold`) o subdominios.  
- **Tabla**: estructura con **columnas tipadas** y **filas**. Puede ser *managed* (gestionada por Databricks) o *external* (ruta externa).  
- **Vista**: consulta guardada que proyecta o filtra datos. Útil para **seguridad a nivel de fila o columna**.  
- **Función**: lógica reusable (SQL/UDF) gobernada por UC.  
- **Modelo**: artefacto de ML registrado y gobernado por UC.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-1.png?raw=true" style="float: right" width="800px"/> 

**Resumen jerárquico**
```
Metastore → Catálogo → Esquema → (Tabla / Vista / Función / Modelo)
```

- Metastore = biblioteca con todos los “libros”.  
- Workspace = tu aula de Databricks.  
- Enlazar = *“esta aula puede usar los libros de esa biblioteca”*.  
Así, desde tu workspace, **ves y usas** catálogos, esquemas y tablas descritos en el metastore.


### 1.3 Reglas Generales a tener en cuenta
- **Mínimo privilegio**: concede solo lo necesario.  
- **Catálogo único de verdad**: metadatos centralizados.  
- **Lineage**: registra orígenes y transformaciones.  
- **Cumplimiento**: soporta auditoría, retención y borrado seguro. 


### 1.4 Arquitectura de Unity Catalog (UC)

La característica principal de Unity Catalog es proporcionarle una forma sencilla de configurar el ACL (nivel de control de acceso) de las tablas, pero también de crear vistas dinámicas basadas en cada permiso individual.



<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/cross_demo_assets/Lakehouse_Demo_Team_architecture_2.png?raw=true" style="float: middle" width="500px">


Con Unity Catalog, tus tablas, usuarios y grupos se definen a nivel de cuenta, entre espacios de trabajo. Ideal para implementar y operar una plataforma Lakehouse en todos tus equipos.

## Bloque 2: Control de Accesos y Seguridad en UC

Gestión de un Metastore, Tipos de Grupos y Tipos de Accesos

- Ahora veremos de manera práctica la creación de grupos.
- Patrón típico: **grupo** (p. ej., *Data Engineers* o *Gobernanza*) como owner de catálogos.  
- **Permisos** frecuentes: `USE CATALOG/SCHEMA`, `SELECT`, `INSERT`, `MODIFY`, `CREATE`, `ALL PRIVILEGES`.  
- Aplica **mínimo privilegio** desde el principio.



### 2.1 Creamos la Jerarquía base de UC (Catalogo-Esquema-Volumnen)

In [0]:
dbutils.widgets.text("CatalogoBloque3","sesion_1")
catalog_name = dbutils.widgets.get("CatalogoBloque3")

In [0]:
spark.sql(f"DROP CATALOG IF EXISTS {catalog_name} CASCADE")


DataFrame[]

In [0]:
# Crear catálogo
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
sql(f"USE CATALOG {catalog_name}")

DataFrame[]

In [0]:
dbutils.widgets.text("EsquemaBloque3","retail_ventas")
schema_name = dbutils.widgets.get("EsquemaBloque3")

In [0]:
sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")

DataFrame[]

### 2.2 Gestión de usuarios y grupos en Unity Catalog

La **gestión de identidades** es clave para aplicar políticas de seguridad en Databricks.  
En lugar de asignar permisos a usuarios individuales, se recomienda trabajar con **grupos**, lo que simplifica y hace más escalable la administración.


### 2.3  Tipos de Grupos: Workspace vs Account
- **Workspace**: los grupos/usuarios creados solo existen dentro de un workspace. Sirven para notebooks o jobs, **pero no son válidos en Unity Catalog**.  
- **Account**: los grupos/usuarios definidos a nivel de cuenta (manual o sincronizados desde Azure AD, Okta, etc.) son los que **reconoce Unity Catalog** y permiten aplicar gobernanza centralizada de datos.  

👉 **Regla general**: para producción y para usar todas las capacidades de UC, siempre define usuarios y grupos a nivel de cuenta.


### 2.4 ¿Hay herencia de permisos en Unity Catalog?

**No existe herencia automática hacia abajo.**  
En Unity Catalog, asignar un permiso en un nivel superior **(catálogo)** no concede automáticamente acceso a los esquemas ni a las tablas que contiene.

Para acceder a una tabla, es obligatorio que el usuario/grupo tenga:

- `USAGE` en el **catálogo**
- `USAGE` en el **esquema**
- `SELECT` (u otro permiso específico) en la **tabla o vista**

Si falta uno solo de esos permisos, el acceso será bloqueado — aunque se haya concedido un permiso en un nivel inferior o superior.

➡️ UC **evalúa jerárquicamente**, pero **no hereda permisos por defecto**. Cada nivel debe ser concedido explícitamente.



### 2.5 Ejercicio Guiado

Mostramos Grupos Nativos

In [0]:
display(spark.sql(f"SHOW GROUPS"))

name,directGroup
admins,null
UC_Data_Engineer,null
users,null
workspace_data_engineer,null


Creamos dos grupos a nivel de workspace:

- workspace_data_analyst
- workspace_data_engineer

In [0]:
%sql
CREATE GROUP workspace_data_engineer;

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-3850940478825438>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE GROUP workspace_data_engineer;\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:152, in SqlMagic.sql(self, line, cell)
    148     raise Exception(
    149         "Cannot run %sql command becau

In [0]:
sql("CREATE GROUP `workspace_data_analyst`")

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-3850940478825439>, line 1
----> 1 sql("CREATE GROUP `workspace_data_analyst`")

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/session.py:736, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    733         _views.append(SubqueryAlias(df._plan, name))
    735 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 736 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    737 if "sql_command_result" in properties:
    738     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1306, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1304     req.user_context.user_id = self._user_id
   1305 req.plan.command.CopyFrom(command

Observamos que los permisos a nivel de Catalogo estan vacios

In [0]:
# Ver permisos a nivel de catálogo
display(spark.sql(f"SHOW GRANTS ON CATALOG {catalog_name}"))


Principal,ActionType,ObjectType,ObjectKey


Podemos visualizar los grupos creados

In [0]:
display(sql("SHOW GROUPS"))

name,directGroup
admins,null
UC_Data_Engineer,null
users,null
workspace_data_engineer,null
workspace_data_analyst,null


Ahora vamos a asignar un permiso a uno de los grupos creados anteriormente.  
Si el grupo **no está sincronizado a nivel de cuenta**, el comando fallará con un error.  

👉 Esto demuestra por qué es esencial gestionar los grupos en el nivel correcto (cuenta) para que Unity Catalog pueda reconocerlos y utilizarlos en la asignación de permisos.


In [0]:
sql(f"GRANT SELECT ON SCHEMA {schema_name} TO `workspace_data_analyst`")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3850940478825445>, line 1
----> 1 sql(f"GRANT SELECT ON SCHEMA {schema_name} TO `workspace_data_analyst`")

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/session.py:736, in SparkSession.sql(self, sqlQuery, args, **kwargs)
    733         _views.append(SubqueryAlias(df._plan, name))
    735 cmd = SQL(sqlQuery, _args, _named_args, _views)
--> 736 data, properties, ei = self.client.execute_command(cmd.command(self._client))
    737 if "sql_command_result" in properties:
    738     df = DataFrame(CachedRelation(properties["sql_command_result"]), self)

File /databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/client/core.py:1306, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1304     req.user_context.user_id = self._user_id
   1305 req.p

Creemos ahora dos grupos a nivel de cuenta: UC_Data_Engineer y UC_Data_Analyst

- Haz click en el icono de tu usuario en la parte superior izquierda
- click en Settings
- identity and access > Groups (Manage)
- Add Group
- Crea 2 grupos: UC_Data_Engineer y UC_Data_Analyst

In [0]:
display(sql("SHOW GROUPS"))

name,directGroup
admins,null
UC_Data_Engineer,null
users,null
workspace_data_engineer,null
workspace_data_analyst,null


In [0]:
%sql
SHOW GRANTS on catalog sesion_1;

Principal,ActionType,ObjectType,ObjectKey


Damos ahora full acces a el grupo data engineer sobre el catalogo

In [0]:
%sql
GRANT ALL PRIVILEGES ON CATALOG sesion_1 TO `UC_Data_Engineer`;

In [0]:
%sql
SHOW GRANTS on catalog sesion_1;

Principal,ActionType,ObjectType,ObjectKey
UC_Data_Engineer,ALL PRIVILEGES,CATALOG,sesion_1


%md
#### Comparativa entre Usuarios y Grupos en Workspace vs Cuenta

La siguiente tabla resume las diferencias clave entre los usuarios y grupos gestionados a nivel de workspace y los gestionados a nivel de cuenta. Es fundamental entender estas diferencias para diseñar una estrategia de seguridad robusta y compatible con Unity Catalog.

| Característica                               | Workspace User/Group     | Account User/Group      |
|----------------------------------------------|--------------------------|-------------------------|
| Visibles solo en un workspace                | ✅ Sí                    | ❌ No (son globales)   |
| Útiles para permisos en notebooks, jobs, etc.| ✅ Sí                    | ✅ Sí                  |
| Se usan para Unity Catalog (datos)           | ❌ No                    | ✅ Sí                  |
| Gobernanza centralizada de datos             | ❌ No                    | ✅ Sí                  |
| Accesibles desde todos los workspaces        | ❌ No                    | ✅ Sí                  |
| Gestión desde Workspace UI                   | ✅ Sí                    | ❌ No (solo desde Account Console o SCIM API) |
| Sincronizables desde IdP (Azure AD, Okta)    | ❌ No                    | ✅ Sí                  |
| Ideal para producción                        | ❌ No                    | ✅ Sí                  |

Como regla general, para entornos productivos y para aprovechar todas las capacidades de Unity Catalog, utiliza siempre usuarios y grupos de cuenta.

### 2.6 Tipos de permisos

#### 2.6.1 A nivel de Metastore / Catálogo / Esquema

- ``USE CATALOG`` / ``USE SCHEMA`` → permite acceder al catálogo/esquema.
- ``CREATE`` → permite crear objetos (esquemas, tablas, vistas, funciones, etc.).
- ``MODIFY`` → modificar objetos existentes.
- ``ALL PRIVILEGES`` → acceso total al objeto

#### 2.6.2 A nivel de Tablas y Vistas

- ``SELECT`` → lectura de datos.
- ``INSERT`` → insertar nuevas filas.
- ``UPDATE`` → actualizar filas existentes.
- ``DELETE`` → eliminar filas.
- ``TRUNCATE`` → vaciar tabla.
- ``REFERENCES`` → crear claves externas (relevante en constraints).
- ``ALL PRIVILEGES`` → todos los anteriores.

**Permisos base de ``USE``**

Para que un grupo pueda ver/usar objetos que cuelgan de un catálogo o esquema, primero necesita USE.

In [0]:
display(spark.sql(f"SHOW GRANTS ON SCHEMA {catalog_name}.{schema_name}"))


Principal,ActionType,ObjectType,ObjectKey
UC_Data_Engineer,ALL PRIVILEGES,CATALOG,sesion_1


#### 2.6.3 Ejemplo Permisos con alumnos

Vamos a crear dos grupos en la clase. Uno de analistas y otro de ingenieros.

Se les dara acceso al catalogo con distintos permisos para ver la gestión.

``Primero crear usuarios a nivel de cuenta``

Haz click en el icono de tu usuario en la parte superior izquierda

click en Settings

identity and access > Users (Manage)

Add User

Creamos los usuarios de clase

- A) Profesor: crea datos y concede accesos iniciales (Analyst ve EU+NA).
- B) Analistas: verifican lectura y fallos de escritura.
- C) Ingenieros: crean/actualizan y publican una vista.
- D) Profesor: revoca NA a Analistas.
- E) Analistas: vuelven a chequear (EU ok, NA falla).

👉 A y D se ejecutan como profesor/owner.
👉 B y E con sesión de Analyst.
👉 C con sesión de Engineer.

**A) Profesor** — Setup de catálogo/esquemas, datos y permisos iniciales

Normalmente utilizaríamos Grupos pero con la cuenta Free Edition no permite agregar usuarios a grupos. Tenemos que agregar los usuarios a nivel de cuenta y luego expecificar los accesos de esos usuarios

In [0]:
catalog_to_drop = "permissions"
spark.sql(f"DROP CATALOG IF EXISTS {catalog_to_drop} CASCADE")

DataFrame[]

In [0]:
# =========================
# Usuarios para el Ejemplo
# =========================

analyst_users  = ["agustin.databricks@gmail.com","possiblepersonalemail@gmail.com"]          # Analistas (EU)
engineer_users = ["251008javiceste@gmail.com"]       # Ingenieros

In [0]:
# =========================
# Creacion de Catalogo>Esquema
# =========================
catalog_perm = "permissions"
schema_bz, schema_sv, schema_gd = "00_bronze", "10_silver", "20_gold"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_perm}")
for s in (schema_bz, schema_sv, schema_gd):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_perm}.{s}")


In [0]:
# =========================
# Datos de demo
# =========================
# Bronze
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_bz}.raw_sales (id INT, raw_line STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_bz}.raw_sales VALUES (1,'1,blade,10'),(2,'2,tower,5')")

# Silver (EU/NA)
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_sv}.sales_EU (id INT, item STRING, qty INT, region STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_sv}.sales_EU VALUES (1,'blade',10,'FR'),(2,'tower',5,'ES')")

spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_sv}.sales_NA (id INT, item STRING, qty INT, region STRING)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_sv}.sales_NA VALUES (1,'blade',8,'US'),(2,'tower',6,'CA')")

# Gold
spark.sql(f"CREATE OR REPLACE TABLE {catalog_perm}.{schema_gd}.sales_summary (item STRING, total_qty INT)")
spark.sql(f"INSERT OVERWRITE {catalog_perm}.{schema_gd}.sales_summary VALUES ('blade',18),('tower',11)")


DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

#### /!\ Dificultad para entrar al workspace compartido /!\

**``una vez dado el acceso les llegara un email a su cuenta para aceptar la participacion``**

Es posible que al aceptar les lleve directamente a mi cuenta. Hay que hacer Log out. Salir del todo

Al entrar nuevamente les saldra para elegir 2 perfiles/workspaces la nueva opcion tendra el nombre del que ha invitado en la participacion.

In [0]:
# =========================
# Grants básicos de navegación
# =========================

#ambos perfiles tendran acceso al catalogo
for u in analyst_users + engineer_users:
    spark.sql(f"GRANT USE CATALOG ON CATALOG {catalog_perm} TO `{u}`")

#perfil analyst tendran acceso a schema silver y gold
for u in analyst_users:
    spark.sql(f"GRANT USE SCHEMA ON SCHEMA {catalog_perm}.{schema_sv} TO `{u}`")  # silver
    spark.sql(f"GRANT USE SCHEMA ON SCHEMA {catalog_perm}.{schema_gd} TO `{u}`")  # gold

#perfil engineer tendra acceso a schema bronze, silver y gold
for u in engineer_users:
    for s in (schema_bz, schema_sv, schema_gd):
        spark.sql(f"GRANT USE SCHEMA ON SCHEMA {catalog_perm}.{s} TO `{u}`")


In [0]:
# =========================
# Permisos de esquema (Engineers construyen en Bronze/Silver)
# =========================

#Damos a ingenieros permiso de escritura de tabla
for s in (schema_bz, schema_sv):
    for u in engineer_users:
        # En PM 1.0, CREATE TABLE en el SCHEMA habilita crear tablas y vistas en ese schema.
        spark.sql(f"GRANT CREATE TABLE ON SCHEMA {catalog_perm}.{s} TO `{u}`")



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

- Analysts: SOLO lectura en EU y NA (tabla a tabla) + lectura en Gold (tabla concreta)
- Engineers: MODIFY (Insert, Update,delete, truncate) en las tablas de bronze y oro


In [0]:
# =========================
# Permisos de tabla/vista (fase inicial: Analysts ven EU+NA en Silver)
# =========================

# Limpiar grants previos de esquema si los hubiera (evita que "ALL TABLES" anule granularidad)
for u in analyst_users:
    # Si en algún momento diste grants a nivel de esquema, revócalos tabla a tabla
    spark.sql(f"REVOKE SELECT ON TABLE {catalog_perm}.{schema_sv}.sales_EU FROM `{u}`")
    spark.sql(f"REVOKE SELECT ON TABLE {catalog_perm}.{schema_sv}.sales_NA FROM `{u}`")
    # Gold: nada que revocar si no diste antes

# Analysts: SOLO lectura en EU y NA (tabla a tabla) + lectura en Gold (tabla concreta)
for u in analyst_users:
    spark.sql(f"GRANT SELECT ON TABLE {catalog_perm}.{schema_sv}.sales_EU TO `{u}`")
    spark.sql(f"GRANT SELECT ON TABLE {catalog_perm}.{schema_sv}.sales_NA TO `{u}`")
    spark.sql(f"GRANT SELECT ON TABLE {catalog_perm}.{schema_gd}.sales_summary TO `{u}`")


# Engineers: RW en Bronze/Silver usando MODIFY (cubre INSERT/UPDATE/DELETE/TRUNCATE)
# 1) Tablas existentes (otorga SELECT + MODIFY por tabla)
engineer_rw_tables = [
    f"{catalog_perm}.{schema_bz}.raw_sales",
    f"{catalog_perm}.{schema_sv}.sales_EU",
    f"{catalog_perm}.{schema_sv}.sales_NA"
]
for u in engineer_users:
    for t in engineer_rw_tables:
        spark.sql(f"GRANT SELECT, MODIFY ON TABLE {t} TO `{u}`")


# 3) Gold solo lectura (tabla existente + futuras)
for u in engineer_users:
    spark.sql(f"GRANT SELECT ON TABLE {catalog_perm}.{schema_gd}.sales_summary TO `{u}`")

print("✅ Bloque de permisos aplicado sin 'ALL TABLES'.")



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:434)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:466)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:757)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:510)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:616)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:643)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:49)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:293)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

**B) Analistas** — Verificación de lectura y fallo de escritura

In [0]:
catalog_perm = "permissions"
schema_bz, schema_sv, schema_gd = "00_bronze", "10_silver", "20_gold"

In [0]:
# Run as Analyst

table_eu = f"{catalog_perm}.{schema_sv}.sales_EU"
table_na = f"{catalog_perm}.{schema_sv}.sales_NA"
table_gd = f"{catalog_perm}.{schema_gd}.sales_summary"

In [0]:
#Select Funciona
  spark.sql(f"SELECT * FROM {table_eu}").show()

+---+-----+---+------+
| id| item|qty|region|
+---+-----+---+------+
|  1|blade| 10|    FR|
|  2|tower|  5|    ES|
+---+-----+---+------+



In [0]:
%sql
-- Select Funciona
SELECT * FROM permissions.10_silver.sales_NA

id,item,qty,region
1,blade,8,US
2,tower,6,CA


In [0]:
#Select Funciona
spark.sql(f"SELECT * FROM {table_gd}").show()

+-----+---------+
| item|total_qty|
+-----+---------+
|blade|       18|
|tower|       11|
+-----+---------+



In [0]:
#Insert No funciona
spark.sql(f"INSERT INTO {table_eu} VALUES (999,'test',1,'DE')")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

**C) Ingenieros** — Crear tabla/vista y operar en Silver

In [0]:
catalog_perm = "permissions"
schema_bz, schema_sv, schema_gd = "00_bronze", "10_silver", "20_gold"

In [0]:
# Run as Engineer

lab_table = "sales_lab"
lab_view  = "vw_items"

# Crear tabla en Silver
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_perm}.{schema_sv}.{lab_table} (
  id INT,
  item STRING,
  qty INT,
  region STRING
)
""")

# Insert/Update/Delete permitidos
spark.sql(f"INSERT INTO {catalog_perm}.{schema_sv}.{lab_table} VALUES (100,'hub',3,'EU'),(101,'blade',2,'ES')")
spark.sql(f"UPDATE {catalog_perm}.{schema_sv}.{lab_table} SET qty = 99 WHERE id = 100")
spark.sql(f"DELETE FROM {catalog_perm}.{schema_sv}.{lab_table} WHERE id = 101")



In [0]:
# Crear vista en Silver
spark.sql(f"""
CREATE OR REPLACE VIEW {catalog_perm}.{schema_sv}.{lab_view} AS
SELECT item, SUM(qty) AS total_qty
FROM (
  SELECT item, qty FROM {catalog_perm}.{schema_sv}.sales_EU
  UNION ALL
  SELECT item, qty FROM {catalog_perm}.{schema_sv}.sales_NA
  UNION ALL
  SELECT item, qty FROM {catalog_perm}.{schema_sv}.{lab_table}
)
GROUP BY item
""")

In [0]:
analyst_users  = ["252210cestean@gmail.com"] 

# Dar acceso a Analysts sobre la vista
for u in analyst_users:
    spark.sql(f"GRANT SELECT ON VIEW {catalog_perm}.{schema_sv}.{lab_view} TO `{u}`")

print("✅ C) Engineer: tabla/vista creadas y operaciones RW comprobadas.")


**D) Profesor** — Revocar acceso a NA para Analysts

In [0]:
# Run as Profesor

for u in analyst_users:
    spark.sql(f"REVOKE SELECT ON TABLE {catalog_perm}.{schema_sv}.sales_NA FROM `{u}`")

# Auditoría rápida
spark.sql(f"SHOW GRANTS ON TABLE {catalog_perm}.{schema_sv}.sales_EU").show(truncate=False)
spark.sql(f"SHOW GRANTS ON TABLE {catalog_perm}.{schema_sv}.sales_NA").show(truncate=False)

print("✅ D) Profesor: Revocado SELECT en sales_NA para Analysts.")


+---------------------+----------+----------+------------------------------+
|Principal            |ActionType|ObjectType|ObjectKey                     |
+---------------------+----------+----------+------------------------------+
|j3671713@gmail.com   |SELECT    |TABLE     |permissions.10_silver.sales_eu|
|u0837436333@gmail.com|MODIFY    |TABLE     |permissions.10_silver.sales_eu|
|u0837436333@gmail.com|SELECT    |TABLE     |permissions.10_silver.sales_eu|
+---------------------+----------+----------+------------------------------+

+---------------------+----------+----------+------------------------------+
|Principal            |ActionType|ObjectType|ObjectKey                     |
+---------------------+----------+----------+------------------------------+
|u0837436333@gmail.com|MODIFY    |TABLE     |permissions.10_silver.sales_na|
|u0837436333@gmail.com|SELECT    |TABLE     |permissions.10_silver.sales_na|
+---------------------+----------+----------+------------------------------

#### 2.6.4 Ver accesos de un perfil sobre un catalogo o schema

In [0]:
%sql
SELECT *
FROM permissions.information_schema.schema_privileges
WHERE grantee = 'usuario@gmail.com';


grantor,grantee,catalog_name,schema_name,privilege_type,is_grantable,inherited_from
251008javiceste@gmail.com,possiblepersonalemail@gmail.com,permissions,10_silver,USE_SCHEMA,NO,NONE
251008javiceste@gmail.com,possiblepersonalemail@gmail.com,permissions,20_gold,USE_SCHEMA,NO,NONE


In [0]:
%sql
show groups with user `251008javiceste@gmail.com`;

name,directGroup
admins,true


### 2.7	Seguridad fila/columna
- Row-level con vistas.
- Column masking con funciones.
- Ejercicio: vista filtrada por región, vista que oculta email.
- (Extra: crear 2 vistas con filtros diferentes y comparar resultados)


#### 2.7.0 Creacion de Ejemplo — Seguridad por filas y columnas

En este caso un alumno liderar como administrador jefe la creacion de tablas y vistas y la gestión de accesos

- A) Chairman: carga el CSV de flights, crea tabla en Silver, define y aplica políticas (row filter + column masks) y concede permisos.
- B) Analistas: confirman que solo ven EU y con PII enmascarada; escritura falla.
- C) Ingenieros: confirman que ven EU+NA y PII sin máscara; además pueden escribir.

**A) Chairman** — Cargar CSV, crear políticas y conceder permisos

**A.1 Elegir usuarios**

In [0]:
analyst_users  = ["252210cestean@gmail.com"]
engineer_users = ["251008javiceste@gmail.com","251022cesteing@gmail.com"]

**A.2 Crear Estructura** Catalogo, Esquema y Volumen

In [0]:
# =========================
# Vars (ajusta usuarios y paths)
# =========================
catalog_ej2 = "chairman"
schema_bz, schema_sv, schema_gd = "00_bronze", "10_silver", "20_gold"
schema_sec = "99_security"  

spark.sql(f"DROP CATALOG IF EXISTS {catalog_ej2} CASCADE")


spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_ej2}")
for s in (schema_bz, schema_sv, schema_gd,schema_sec):
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_ej2}.{s}")

In [0]:
%sql
-- Crea el volume para aterrizar ficheros
CREATE VOLUME IF NOT EXISTS chairman.00_bronze.flights_vol;


**A.3 Subir el csv a el volumen y cargar**

**A.4 Crear la tabla**

In [0]:
csv_path = f"/Volumes/{catalog_ej2}/{schema_bz}/flights_vol/flights.csv"
df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(csv_path))

display(df.limit(2))

flight_id,market,origin,destination,pax_name,pax_email,credit_card,fare_eur,status
1,EU,BCN,CDG,Ana Lopez,ana.lopez@gmail.com,4111111111111111,120,ON_TIME
2,EU,MAD,FCO,Luis Perez,luis.perez@gmail.com,4222222222222222,150,DELAYED


In [0]:
# Normaliza tipos (por si inferSchema deja algo como string numérico)
from pyspark.sql.functions import col
df = (df
      .withColumn("fare_eur", col("fare_eur").cast("decimal(10,2)"))
      .withColumn("flight_id", col("flight_id").cast("string")))

table_flights = f"{catalog_ej2}.{schema_sv}.flights"

df.write.mode("overwrite").saveAsTable(table_flights)


**A.5 Exploracin Tabla**

Vemos que las columnas, origines, destinos, nombre, email, tarjeta credito.... 

El ``objetivo va a ser``:
- ocultar todo lo que sea mercado = NA para analistas 
- encriptar la informacion sensible (email, tarjeta credito...)

todo ello sin crear nuevas tablas. dependiendo de que usuario accede, tendra disponible una informacion u otra

In [0]:
spark.sql(f"""
          select *
            from chairman.`10_silver`.flights""").show()

spark.sql(f"""
          select distinct(market)
            from chairman.`10_silver`.flights""").show()


+---------+------+------+-----------+---------------+--------------------+----------------+--------+---------+
|flight_id|market|origin|destination|       pax_name|           pax_email|     credit_card|fare_eur|   status|
+---------+------+------+-----------+---------------+--------------------+----------------+--------+---------+
|        1|    EU|   BCN|        CDG|      Ana Lopez| ana.lopez@gmail.com|4111111111111111|  120.00|  ON_TIME|
|        2|    EU|   MAD|        FCO|     Luis Perez|luis.perez@gmail.com|4222222222222222|  150.00|  DELAYED|
|        3|    EU|   BCN|        LHR|    Maria Gomez|maria.gomez@gmail...|4333333333333333|   95.00|CANCELLED|
|        4|    EU|   MAD|        FRA|    Javier Ruiz|javier.ruiz@gmail...|4444444444444444|  200.00|  ON_TIME|
|        5|    EU|   LIS|        AMS|    Pedro Silva|pedro.silva@gmail...|4555555555555555|  110.00|  DELAYED|
|        6|    EU|   BCN|        ORY|   Elena Torres|elena.torres@gmai...|4666666666666666|  130.00|CANCELLED|
|

**A.5 Damos accesos a Nuevo Catalogo y esquemas**

Damos accesos a catalogop y esquemas a analista e ingenieros.

En este punto solo tendran acceso a ver el catalogo y esquema. Aun no les habremos dado acceso a la tabla, por lo que les aparecera el schema vacio

In [0]:
# =========================
# Grants mínimos de navegación
# =========================
for u in analyst_users + engineer_users:
    spark.sql(f"GRANT USE CATALOG ON CATALOG {catalog_ej2} TO `{u}`")
    spark.sql(f"GRANT USE SCHEMA  ON SCHEMA  {catalog_ej2}.{schema_sv} TO `{u}`")
    spark.sql(f"GRANT USE SCHEMA  ON SCHEMA  {catalog_ej2}.{schema_sec} TO `{u}`")

**A.6 Construimos nueva lista de engineers**

Para seguir con los accesos vamos a usar el ejemplo de que contamos con un csv para ello, al cual añadimos los usuarios que hemos defindido en el apartado A.1


Si engineer_users = ["u0837...@gmail.com"], entonces engineers_csv queda como:
u0837...@gmail.com → y lo inyectamos en SQL como ('u0837...@gmail.com').

Si hubiera varios, quedaría ('eng1@...','eng2@...').

In [0]:
engineers_csv = "', '".join(engineer_users)


In [0]:
display(engineers_csv)

"251008javiceste@gmail.com', '251022cesteing@gmail.com"

#### 2.7.1 Row level access control 

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/acls/table_uc_rls.png?raw=true" width="200" style="float: right; margin-top: 20; margin-right: 20" alt="databricks-demos"/>

La seguridad a nivel de fila le permite ocultar automáticamente un subconjunto de sus filas en función de quién intente consultarlas, sin tener que mantener copias separadas de sus datos.

Un caso de uso típico sería filtrar las filas en función de su país o unidad de negocio: solo verá los datos (transacciones financieras, pedidos, información de clientes...) pertenecientes a su región, lo que le impedirá acceder a todo el conjunto de datos.

💡 Aunque este filtro se puede aplicar a nivel de usuario/principal, se recomienda implementar políticas de acceso utilizando grupos.
<br style="clear: both"/>

##### 2.7.1.1 Definir la regla de acceso

Para declarar una regla de control de acceso, deberá crear una función SQL que devuelva un valor **booleano**.
Unity Catalog ocultará la fila si la función devuelve `False`.

Dentro de la función SQL, puede definir diferentes condiciones e implementar una lógica compleja para crear este valor de retorno booleano. (Por ejemplo:  `IF(condición)-THEN(vista)-ELSE`)

Aquí, aplicaremos la siguiente lógica:

- Tipo: función SQL.
- input: recibe el valor de la columna market y devuelve TRUE/FALSE.
- Lógica:
  - Si el usuario actual (CURRENT_USER()) está en la lista de engineers → TRUE (ve todas las filas).
  - Si no es engineer → solo TRUE cuando market='EU'.

La funcion queda almacenada en el schema 99_security

In [0]:
%sql
CREATE OR REPLACE FUNCTION chairman.99_security.rf_only_eu_or_engineers(market STRING)
RETURNS BOOLEAN
RETURN
  CASE
    -- Si el usuario está en la lista de engineers → acceso total (TRUE SIEMPRE)
    WHEN CURRENT_USER() IN ('251008javiceste@gmail.com', '251022cesteing@gmail.com') THEN TRUE

    -- Si NO es engineer → solo permitimos EU
    WHEN market = 'EU' THEN TRUE

    -- En cualquier otro caso → DENEGADO
    ELSE FALSE
  END;


In [0]:
%sql

--Lamentablemente en la version free no podemos usar este tipo de listas, ni array... hay que pasarlo como en la celda anterior

--CREATE OR REPLACE FUNCTION chairman.99_security.rf_only_eu_or_engineers(market STRING)
--RETURNS BOOLEAN
-- --RETURN
--   CASE
--     -- Si el usuario está en la lista de engineers → acceso total (TRUE SIEMPRE)
--     WHEN CURRENT_USER() IN ('{engineers_csv}') THEN TRUE

--     -- Si NO es engineer → solo permitimos EU
--     WHEN market = 'EU' THEN TRUE

--     -- En cualquier otro caso → DENEGADO
--     ELSE FALSE
--   END;



##### 2.7.1.2 Aplicar la regla de acceso

Una vez declarada nuestra función de regla, solo queda aplicarla a una tabla y verla en acción.
Basta con un simple `SET ROW FILTER` seguido de una llamada a la función.

**Nota: si esto falla, asegúrese de que está utilizando un clúster compartido.**

In [0]:
%sql
ALTER TABLE chairman.10_silver.flights 
SET ROW FILTER chairman.99_security.rf_only_eu_or_engineers
ON (market);

In [0]:
%sql
select distinct(market)
from chairman.`10_silver`.flights

market
EU
NA


**3.7.1.3 Damos accesos a la tabla**

In [0]:
# Analysts: solo lectura
for u in analyst_users:
    spark.sql(f"GRANT SELECT ON table `chairman`.`10_silver`.`flights` TO `{u}`")

# Engineers: lectura + modificación (INSERT/UPDATE/DELETE/TRUNCATE/MERGE)
for u in engineer_users:
    spark.sql(f"GRANT SELECT, MODIFY ON TABLE `chairman`.`10_silver`.`flights` TO `{u}`")

###### Resultado de la política de seguridad

**Analyst ejecuta → `SELECT * FROM flights`**

| Fila                 | Evaluación función                       | Resultado |
|----------------------|-------------------------------------------|-----------|
| (`market = 'NA'`, ...) | `FALSE (no EU)` y tampoco en engineers   | ❌ Filtrada |
| (`market = 'EU'`, ...) | `TRUE (es EU)`                          | ✅ Visible |

<br>

**Engineer ejecuta lo mismo → `SELECT * FROM flights`**

| Fila                 | Evaluación función                       | Resultado |
|----------------------|-------------------------------------------|-----------|
| Cualquier market      | `CURRENT_USER() IN engineers → TRUE`     | ✅ Todas visibles |



#### 2.7.2 Column Level access control 

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/acls/table_uc_cls.png?raw=true" width="200" style="float: right; margin-top: 20; margin-right: 20; margin-left: 20" alt="databricks-demos"/>

Del mismo modo, el control de acceso a nivel de columna le ayuda a ocultar o anonimizar los datos que se encuentran en determinadas columnas de su tabla, dependiendo del usuario o del servicio principal que intente acceder a ellos. Esto se utiliza normalmente para ocultar o eliminar información confidencial de sus usuarios finales (correo electrónico, número de la seguridad social, etc.).

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=1330931038747594&notebook=%2F01-Row-Column-access-control&demo_name=uc-01-acl&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-01-acl%2F01-Row-Column-access-control&version=1">

##### 2.7.2.1 Definir la regla de acceso
- Tipo: función SQL de **enmascaramiento dinámico**.
- Firma: recibe el valor real de la columna (cc o email) y devuelve una versión **visible o enmascarada**.
- Lógica:
  - Si `CURRENT_USER()` está en la lista de *engineers* → devuelve el valor **real** (sin máscara).
  - Si el dato es NULL → devuelve NULL (no modifica nada).
  - Si es un usuario normal (analyst) → devuelve una **versión parcialmente enmascarada**:
    - `mask_credit_card` → solo muestra los **4 últimos dígitos**.
    - `mask_email` → solo muestra los **2 primeros caracteres** y oculta el resto.


In [0]:
%sql
CREATE OR REPLACE FUNCTION chairman.99_security.mask_email(e STRING) RETURNS STRING
RETURN CASE
  WHEN CURRENT_USER() IN ('...engineers_csv...') THEN e        -- engineer ve real
  WHEN e IS NULL THEN NULL
  ELSE CONCAT(SUBSTR(e, 1, 2), '***@***')                      -- analyst ve máscara
END


In [0]:
%sql
CREATE OR REPLACE FUNCTION chairman.99_security.mask_credit_card(cc STRING) RETURNS STRING
RETURN CASE
  WHEN CURRENT_USER() IN ('...engineers_csv...') THEN cc       -- engineer ve real
  WHEN cc IS NULL THEN NULL
  ELSE CONCAT('**** **** **** ', RIGHT(cc, 4))                   -- analyst ve solo 4 últimos
END



##### 2.7.2.2 Aplicamos la política de enmascaracion de columnas a la tabla

Aquí acoplas la función rf_eu_only a la tabla, indicando que se evalúe sobre la columna market.

Efecto: toda lectura de esa tabla pasará por ese filtro dinámico.



In [0]:
%sql
select *
from chairman.`10_silver`.flights

flight_id,market,origin,destination,pax_name,pax_email,credit_card,fare_eur,status
1,EU,BCN,CDG,Ana Lopez,ana.lopez@gmail.com,4111111111111111,120.00,ON_TIME
2,EU,MAD,FCO,Luis Perez,luis.perez@gmail.com,4222222222222222,150.00,DELAYED
3,EU,BCN,LHR,Maria Gomez,maria.gomez@gmail.com,4333333333333333,95.00,CANCELLED
4,EU,MAD,FRA,Javier Ruiz,javier.ruiz@gmail.com,4444444444444444,200.00,ON_TIME
5,EU,LIS,AMS,Pedro Silva,pedro.silva@gmail.com,4555555555555555,110.00,DELAYED
6,EU,BCN,ORY,Elena Torres,elena.torres@gmail.com,4666666666666666,130.00,CANCELLED
7,NA,JFK,MIA,John Smith,john.smith@gmail.com,4777777777777777,200.00,CANCELLED
8,NA,ORD,LAX,Susan Miller,susan.miller@gmail.com,4888888888888888,180.00,ON_TIME
9,NA,BOS,YYZ,Robert Brown,robert.brown@gmail.com,4999999999999999,140.00,DELAYED
10,NA,MIA,ATL,Emily Davis,emily.davis@gmail.com,4000000000000000,160.00,ON_TIME


In [0]:
%sql
-- Vista para Analysts: solo EU + PII enmascarada con tus UDFs
CREATE OR REPLACE TABLE chairman.10_silver.vw_flights_analyst AS
SELECT
  flight_id,
  market,
  origin,
  destination,
  fare_eur,
  status,
  pax_name,
  chairman.99_security.mask_email(pax_email) AS pax_email,
  chairman.99_security.mask_credit_card(credit_card) AS pax_credit_card
FROM chairman.10_silver.flights
WHERE chairman.99_security.rf_only_eu_or_engineers(market);   -- filtra filas con tu UDF


num_affected_rows,num_inserted_rows


In [0]:
%sql
select *
from chairman.`10_silver`.vw_flights_analyst

flight_id,market,origin,destination,fare_eur,status,pax_name,pax_email,pax_credit_card
1,EU,BCN,CDG,120.00,ON_TIME,Ana Lopez,an***@***,**** **** **** 1111
2,EU,MAD,FCO,150.00,DELAYED,Luis Perez,lu***@***,**** **** **** 2222
3,EU,BCN,LHR,95.00,CANCELLED,Maria Gomez,ma***@***,**** **** **** 3333
4,EU,MAD,FRA,200.00,ON_TIME,Javier Ruiz,ja***@***,**** **** **** 4444
5,EU,LIS,AMS,110.00,DELAYED,Pedro Silva,pe***@***,**** **** **** 5555
6,EU,BCN,ORY,130.00,CANCELLED,Elena Torres,el***@***,**** **** **** 6666
7,NA,JFK,MIA,200.00,CANCELLED,John Smith,jo***@***,**** **** **** 7777
8,NA,ORD,LAX,180.00,ON_TIME,Susan Miller,su***@***,**** **** **** 8888
9,NA,BOS,YYZ,140.00,DELAYED,Robert Brown,ro***@***,**** **** **** 9999
10,NA,MIA,ATL,160.00,ON_TIME,Emily Davis,em***@***,**** **** **** 0000


**A.8 Damos permisos para entrar a la tabla (GRANT)**

Importante: las políticas (row filter / masks) no sustituyen los GRANT.

Primero necesitas el derecho a leer (SELECT),

y después las políticas modulan lo que ves (filas/columnas).

Para escribir, usamos MODIFY (engloba las operaciones de DML usuales).

In [0]:
# Analysts: solo lectura
for u in analyst_users:
    spark.sql(f"GRANT SELECT ON VIEW chairman.10_silver.vw_flights_analyst TO `{u}`")

# Engineers: lectura + modificación (INSERT/UPDATE/DELETE/TRUNCATE/MERGE)
for u in engineer_users:
    spark.sql(f"GRANT SELECT, MODIFY ON TABLE {table_flights} TO `{u}`")


In [0]:
%sql
-- ALTER TABLE ... DROP ROW FILTER permissions.99_security.vw_flights_analyst


## Bloque extra: Ejercicios EXTRA de ACCESOS

- Engineer → u0837436333@gmail.com
- Analyst → j3671713@gmail.com
- Catálogo → permissions
- Esquemas → 00_bronze, 10_silver, 20_gold

**Ejercicio 1: Catálogo y esquemas**

Crea el catálogo permissions y los esquemas 00_bronze, 10_silver, 20_gold.

In [0]:
%sql
-- %sql
-- TODO: crea el catálogo y los esquemas
CREATE CATALOG IF NOT EXISTS permissions;

CREATE SCHEMA IF NOT EXISTS permissions.`00_bronze`;
CREATE SCHEMA IF NOT EXISTS permissions.`10_silver`;
CREATE SCHEMA IF NOT EXISTS permissions.`20_gold`;

-- Verificación
SHOW SCHEMAS IN permissions;


databaseName
00_bronze
10_silver
20_gold
default
information_schema


**Ejercicio 2: Da accesos de USO a catalogo y esquemas (todos)**

In [0]:
%sql
-- %sql (opcional) grants de navegación por email
GRANT USE CATALOG ON CATALOG permissions TO `u0837436333@gmail.com`;
GRANT USE CATALOG ON CATALOG permissions TO `j3671713@gmail.com`;

GRANT USE SCHEMA ON SCHEMA permissions.`00_bronze` TO `u0837436333@gmail.com`;
GRANT USE SCHEMA ON SCHEMA permissions.`10_silver` TO `u0837436333@gmail.com`;
GRANT USE SCHEMA ON SCHEMA permissions.`20_gold` TO `u0837436333@gmail.com`;

GRANT USE SCHEMA ON SCHEMA permissions.`10_silver` TO `j3671713@gmail.com`;
GRANT USE SCHEMA ON SCHEMA permissions.`20_gold` TO `j3671713@gmail.com`;


**Ejercicio 3: Crea dos Tablas en Silver**

In [0]:
%sql
CREATE OR REPLACE TABLE permissions.`10_silver`.ops_flights (
  flight_id   STRING,
  flight_date DATE,
  market      STRING,     -- EU / NA
  origin      STRING,     -- IATA
  destination STRING,     -- IATA
  sched_dep   TIMESTAMP,  -- salida programada
  actual_dep  TIMESTAMP,  -- salida real
  delay_min   INT,        -- minutos de retraso (NULL si cancelado)
  status      STRING      -- ON_TIME / DELAYED / CANCELLED
);

INSERT OVERWRITE permissions.`10_silver`.ops_flights VALUES
-- EU
('VY1001', DATE'2025-09-20', 'EU', 'BCN', 'CDG', TIMESTAMP'2025-09-20 08:00:00', TIMESTAMP'2025-09-20 08:02:00',  2,  'ON_TIME'),
('VY1002', DATE'2025-09-20', 'EU', 'MAD', 'FCO', TIMESTAMP'2025-09-20 09:30:00', TIMESTAMP'2025-09-20 09:55:00', 25, 'DELAYED'),
('VY1003', DATE'2025-09-20', 'EU', 'BCN', 'LHR', TIMESTAMP'2025-09-20 10:15:00', TIMESTAMP'2025-09-20 10:15:00', 0,  'ON_TIME'),
('VY1004', DATE'2025-09-21', 'EU', 'MAD', 'FRA', TIMESTAMP'2025-09-21 07:45:00', NULL,                           NULL,'CANCELLED'),
-- NA
('VY2001', DATE'2025-09-20', 'NA', 'JFK', 'MIA', TIMESTAMP'2025-09-20 11:00:00', TIMESTAMP'2025-09-20 12:10:00', 70, 'DELAYED'),
('VY2002', DATE'2025-09-20', 'NA', 'ORD', 'LAX', TIMESTAMP'2025-09-20 13:20:00', TIMESTAMP'2025-09-20 13:28:00', 8,  'ON_TIME'),
('VY2003', DATE'2025-09-21', 'NA', 'BOS', 'YYZ', TIMESTAMP'2025-09-21 06:50:00', TIMESTAMP'2025-09-21 07:05:00', 15, 'DELAYED'),
('VY2004', DATE'2025-09-21', 'NA', 'MIA', 'ATL', TIMESTAMP'2025-09-21 16:10:00', TIMESTAMP'2025-09-21 16:10:00', 0,  'ON_TIME');

SELECT * FROM permissions.`10_silver`.ops_flights;


In [0]:
%sql
-- Solución (profesor)
CREATE OR REPLACE TABLE permissions.`10_silver`.pax_bookings (
  booking_id   STRING,
  flight_id    STRING,
  pax_name     STRING,
  pax_email    STRING,
  credit_card  STRING,         -- nº completo -> para enmascarar
  fare_eur     DECIMAL(10,2),
  seat         STRING
);

INSERT OVERWRITE permissions.`10_silver`.pax_bookings VALUES
-- EU
('BK1001','VY1001','Ana Lopez',   'ana.lopez@gmail.com',   '4111111111111111', 120.00,'12A'),
('BK1002','VY1002','Luis Perez',  'luis.perez@gmail.com',  '4222222222222222', 150.00,'22C'),
('BK1003','VY1003','Maria Gomez', 'maria.gomez@gmail.com', '4333333333333333',  95.00,'07F'),
('BK1004','VY1004','Javier Ruiz', 'javier.ruiz@gmail.com', '4444444444444444', 200.00,'14D'),
-- NA
('BK2001','VY2001','John Smith',  'john.smith@gmail.com',  '4555555555555555', 210.00,'10B'),
('BK2002','VY2002','Susan Miller','susan.miller@gmail.com','4666666666666666', 180.00,'19A'),
('BK2003','VY2003','Robert Brown','4777777777777777',      140.00,'03C'),
('BK2004','VY2004','Emily Davis', '4888888888888888',      160.00,'21E');

SELECT * FROM permissions.`10_silver`.pax_bookings;


**Ejercicio 4: Crear schema de vistas seguras y controlar quién puede verlo**

Crea un esquema para vistas de consumo: permissions.30_secure.

Concede solo USE SCHEMA a quienes vayan a consultar vistas (p. ej., Analyst/Engineer) y deja al resto sin acceso.

In [0]:
%sql

-- Solución (profesor)
CREATE SCHEMA IF NOT EXISTS permissions.`30_secure`;

-- Define quién puede entrar al schema de vistas
GRANT USE SCHEMA ON SCHEMA permissions.`30_secure` TO `j3671713@gmail.com`;  -- Analyst
GRANT USE SCHEMA ON SCHEMA permissions.`30_secure` TO `u0837436333@gmail.com`; -- Engineer

-- (Opcional) revoca si diste acceso antes a alguien que no deba
-- REVOKE USE SCHEMA ON SCHEMA permissions.`30_secure` FROM `<otro@dominio>`;




**Ejercicio 5: Crear una función de enmascarado (en schema seguro de funciones)**

Crea en permissions.99_security una función mask_credit_card(cc STRING) que muestre solo los 4 últimos dígitos.

In [0]:
%sql
-- Solución (profesor)
CREATE OR REPLACE FUNCTION permissions.`99_security`.mask_credit_card(cc STRING) RETURNS STRING
RETURN CASE
  WHEN cc IS NULL THEN NULL
  ELSE CONCAT('**** **** **** ', RIGHT(cc, 4))
END;

SHOW FUNCTIONS IN permissions.`99_security`;



**Ejercicio 6: Crear una vista aplicando la función de enmascarado**

En permissions.30_secure, crea una vista vw_ops_pax_secure que:

- Haga JOIN entre pax_bookings y ops_flights.
- Use mask_credit_card(credit_card) para enmascarar PII.
- (Opcional) filtre a market='EU' para un ejemplo de RLS simulado.

In [0]:
%sql
-- Solución (profesor)
CREATE OR REPLACE VIEW permissions.`30_secure`.vw_ops_pax_secure AS
SELECT
  b.booking_id,
  b.flight_id,
  f.market,
  f.origin, f.destination,
  b.pax_name,
  b.pax_email,
  permissions.`99_security`.mask_credit_card(b.credit_card) AS credit_card_masked,
  b.fare_eur,
  f.status,
  f.flight_date
FROM permissions.`10_silver`.pax_bookings b
JOIN permissions.`10_silver`.ops_flights   f
  ON b.flight_id = f.flight_id
-- Descomenta si quieres limitar a EU:
-- WHERE f.market = 'EU'
;

SELECT * FROM permissions.`30_secure`.vw_ops_pax_secure;



**Ejercicio 7: Conceder accesos a la vista y a las tablas**

- Da SELECT sobre la vista al Analyst.
- Da SELECT, MODIFY sobre tablas base al Engineer (y SELECT sobre la vista si quieres).

In [0]:
%sql
-- Solución (profesor)
-- Navegación necesaria
GRANT USE CATALOG ON CATALOG permissions TO `j3671713@gmail.com`;
GRANT USE CATALOG ON CATALOG permissions TO `u0837436333@gmail.com`;
GRANT USE SCHEMA  ON SCHEMA  permissions.`10_silver` TO `j3671713@gmail.com`;
GRANT USE SCHEMA  ON SCHEMA  permissions.`10_silver` TO `u0837436333@gmail.com`;

-- Vista segura (consumo)
GRANT SELECT ON VIEW permissions.`30_secure`.vw_ops_pax_secure TO `j3671713@gmail.com`;  -- Analyst
GRANT SELECT ON VIEW permissions.`30_secure`.vw_ops_pax_secure TO `u0837436333@gmail.com`;-- Engineer (opcional)

-- Tablas base para ingeniería
GRANT SELECT, MODIFY ON TABLE permissions.`10_silver`.ops_flights  TO `u0837436333@gmail.com`;
GRANT SELECT, MODIFY ON TABLE permissions.`10_silver`.pax_bookings TO `u0837436333@gmail.com`;


**Ejercicio 8: Crear una tabla en Gold con MERGE y GROUP BY**

- Construye permissions.20_gold.ops_kpis agregada por fecha, mercado y estado.
- Crea primero la tabla vacía con PK lógica (flight_date, market, status).
- Usa MERGE INTO desde una subconsulta agregada sobre ops_flights.

In [0]:
%sql
-- 8.1) Crear la tabla objetivo en Gold (si no existe)
CREATE TABLE IF NOT EXISTS permissions.`20_gold`.ops_kpis (
  flight_date DATE,
  market      STRING,
  status      STRING,   -- ON_TIME / DELAYED / CANCELLED
  flights     BIGINT,
  avg_delay   DOUBLE
);

-- 8.2) Fuente agregada (puedes convertirla en VIEW si quieres)
--     KPIs por fecha, mercado, status. Usamos delay_min solo donde aplica.
MERGE INTO permissions.`20_gold`.ops_kpis AS tgt
USING (
  SELECT
    flight_date,
    market,
    COALESCE(status,'UNKNOWN') AS status,
    COUNT(*)                             AS flights,
    AVG(CASE WHEN delay_min IS NOT NULL THEN delay_min ELSE 0 END) AS avg_delay
  FROM permissions.`10_silver`.ops_flights
  GROUP BY flight_date, market, COALESCE(status,'UNKNOWN')
) AS src
ON  tgt.flight_date = src.flight_date
AND tgt.market      = src.market
AND tgt.status      = src.status
WHEN MATCHED THEN
  UPDATE SET tgt.flights = src.flights, tgt.avg_delay = src.avg_delay
WHEN NOT MATCHED THEN
  INSERT (flight_date, market, status, flights, avg_delay)
  VALUES (src.flight_date, src.market, src.status, src.flights, src.avg_delay)
;

-- 8.3) Ver resultado
SELECT * FROM permissions.`20_gold`.ops_kpis ORDER BY flight_date, market, status;


### Chequeos del Profesor

Con sesión de Analyst:

- Lee sales_EU (debe funcionar).
- Intenta insertar en sales_EU (debe fallar).
- Prueba leer sales_NA (si no le has dado acceso, debe fallar).

In [0]:
# Solución (profesor) — ejecutar como Analyst
table_eu = "permissions.10_silver.sales_EU"
table_na = "permissions.10_silver.sales_NA"

spark.sql(f"SELECT * FROM {table_eu}").show()

try:
    spark.sql(f"INSERT INTO {table_eu} VALUES (999,'test',1,'DE')")
    print("❌ No esperado: Analyst pudo insertar.")
except Exception as e:
    print("✅ Esperado: Analyst no puede insertar ->", str(e)[:200])

try:
    spark.sql(f"SELECT * FROM {table_na} LIMIT 1").show()
    print("⚠️ Analyst aún puede leer NA (revisa grants si no querías permitirlo).")
except Exception as e:
    print("✅ Esperado: Analyst no puede leer sales_NA ->", str(e)[:200])

Con sesión de Engineer:

- Crea sales_lab en Silver.
- Realiza INSERT/UPDATE/DELETE.
- Verifica el resultado.

In [0]:
%sql
-- Solución (profesor) — ejecutar como Engineer
CREATE TABLE IF NOT EXISTS permissions.`10_silver`.sales_lab (
  id INT, item STRING, qty INT, region STRING
);

INSERT INTO permissions.`10_silver`.sales_lab
VALUES (100,'hub',3,'EU'), (101,'blade',2,'ES');

UPDATE permissions.`10_silver`.sales_lab SET qty = 99 WHERE id = 100;
DELETE FROM permissions.`10_silver`.sales_lab WHERE id = 101;

SELECT * FROM permissions.`10_silver`.sales_lab;



En este ejercicio vamos a:

Crear un esquema y una tabla sensible en Unity Catalog
Crear grupos y usuarios
Asignar permisos diferenciados a distintos perfiles
Validar los accesos mediante consultas

# Unity Catalog: Data Discovery y Linaje

## Bloque 3: AI generated Documentation + Tagging

#### 3.1 Documentación generada por IA

<img src="https://raw.githubusercontent.com/jmartinezceste/251101Course_UC_Ceste/main/photos/ai_genera_documentacion.png" width="700">


- En el *Data Explorer* (Unity Catalog), puedes hacer clic en **"Generate documentation with AI"**.
- Databricks analiza automáticamente nombres de columnas, tipos de datos y patrones.
- Genera descripciones en lenguaje natural tanto para **la tabla como para cada columna**.
- Permite **editar, aceptar y guardar** el resultado directamente como documentación oficial del objeto.
- Facilita **el descubrimiento de datos** y evita depender del “conocimiento tribal”.


#### 3.2 Tagging (etiquetado de datos)


Coloca Tags para **facilitar la busqueda de datos y definir politicas**


- Puedes asignar **tags** a catálogos, esquemas, tablas y columnas.
- Útil para clasificar PII, datos financieros, confidenciales, internos, etc.
- Databricks puede incluso **sugerir tags automáticamente usando IA**.
- Estos tags pueden ser usados para **automatizar políticas** (ej. aplicar máscara si `tag = PII`).
- Luego es posible buscar datasets por tag → *“ver todos los datos certificados y financieros”*.


##### 3.2.1 ¿Cómo crear un Governed Tag en Unity Catalog?

1. Ir a **Catalog → Governance → Governed Tags**  
2. Clic en **“Create Tag”**  
3. Definir:  
   - Creamos 2 tags:
      - Business_domain: Opciones (Finance, Marketing, Logistics)
      - Bunsiness_sensibility: Opciones (High, Medium, Low)
      

Una vez creado, este tag estará disponible para asignarlo a **catálogos, esquemas, tablas o columnas**.


**Aqui voy a hacer ejemplo de crear tags, darseloas a tablas, catalogos. Ver luego como podemos verlos...**

## Bloque 4: Tablas - Explorando todos sus tabs

Ahora Naveguemos por toda la información disponible en UC referida a una tabla que hayamos creado

### Información clave en las pestañas de una tabla dentro de **Unity Catalog**

### 1. Overview (lo esencial para entender y gobernar el activo)
- **Descripción generada por IA**: útil para documentación automática si no hay descripción manual.
- **Owner**: responsable funcional o técnico de la tabla (clave para gobernanza y aprobaciones).
- **Tipo de objeto**: tabla managed, external, view, volume, etc.
- **Tags / clasificadores**: perfectos para filtros por dominio, confidencialidad (`PII`, `Finance`, `Bronze`, etc.), cumplimiento normativo.

### 2. Sample Data
- Vista rápida de las primeras filas para *validación visual* sin necesidad de ejecutar un `SELECT`.

### 3. Details (profundamente útil para ingenieros)
- **Storage Location**: ruta parquet real → esencial para troubleshooting y análisis de coste.
- **Format**: indica si es Delta, CSV, parquet, view, etc.
- **Created by / Updated by**: quién hizo los cambios.
- **Updated at**: timestamp exacto de última modificación.  
  → *Si necesitas volver atrás: `SELECT * FROM tabla VERSION AS OF <n>` o `TIMESTAMP AS OF`.*
- **Table Properties**: configuración avanzada (`delta.enableChangeDataFeed`, `delta.logRetention`, etc.)

### 4. Permissions
- Control de acceso a nivel de catálogo ESQUEMA/tabla.
- Muestra si hay **políticas de Row-Level Security o Column Masking activas**.
- Indispensable para cumplimiento y data sharing controlado.

### 5. History
- Auditoría completa **automática e inmutable** de todas las operaciones (CREATE, ALTER, OPTIMIZE…).
- Similar a un “Git de tablas”.
- Puedes ver eventos pasados, incluso antes de acceder vía Time Travel → ideal para reversión con precisión.

### 6. Lineage
- **Traza visual completa del flujo de datos — upstream y downstream**.
- Indica qué notebooks, workflows, tables o dashboards consumen y alimentan esta tabla.
- Fundamental para impact analysis antes de modificar estructuras o reglas.

### 7. Insights
- Perfilado automático de datos: top valores, distribución, valores nulos, cardinalidad, outliers.
- Acelera validaciones y detección de anomalías sin SQL manual.

### 8. Quality (si está activo en tu workspace)
- Registra reglas de calidad configuradas sobre la tabla.
- Muestra porcentajes de éxito/fallo, tendencia de calidad en el tiempo.
- Muy útil para detección proactiva y alertas automáticas.




## Bloque 5: Lineage


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/lineage/uc-lineage-slide.png?raw=true" style="float:right; margin-left:10px" width="700"/>

### 5.1 Linaje en Unity Catalog

Unity Catalog captura el linaje de datos en tiempo de ejecución para cualquier operación de tabla a tabla ejecutada en un clúster Databricks o endpoint SQL. Lineage opera en todos los lenguajes (SQL, Python, Scala y R) y puede visualizarse en el Explorador de datos casi en tiempo real, y también recuperarse a través de REST API.

Lineage está disponible en dos niveles de granularidad:
- Tablas
- Columnas: ideal para rastrear dependencias GDPR

Lineage tiene en cuenta las ACL de tabla presentes en Unity Catalog. Si a un usuario no se le permite ver una tabla en un punto determinado del gráfico, su información se redacta, pero aún puede ver que una tabla ascendente o descendente está presente.

###5.2 Trabajar con Lineage

Es importante por:

- Compliance:
  - Requisitos Regulatorios pueden ser verificados con Linaje
  - Sirve para traquear el rastro de datos sensibles

- Data Discovery:
  - Entender el contexto y la integridad del dato antes de usarlo en analisis.
  - Prevenir duplicar esfuerzos

- Observability
  - Analizar el impacto de modificaciones a reportes aguas abajo
  - Rastrear el problemas y discrepancias hacia origen


### 5.3 Ejercicios

In [0]:
# Modo Python
dbutils.notebook.run("/Workspace/Users/251008javiceste@gmail.com/lineage_set_up_def", timeout_seconds=0)

---

**1. Evolución de estructura / tipo de dato**

Nos informan de que el campo presupuesto_mensual en la tabla GOLD ``dim_restaurant`` está redondeado respecto a versiones previas. 

- ¿Puedes comprobar si en alguna capa intermedia ha cambiado el tipo de dato? Y si sí, ¿desde qué capa viene ese cambio?


Respuesta: 

En la primera tabla de dim_restaurant de bronce se ve que el origen de tipo de columna era DOUBLE y pasa a INT

---

**2. ¿De dónde viene realmente este campo?**

En la tabla GOLD ``fact_menu_sales`` hay una columna llamada margen. 
- ¿Puedes navegar el linaje y explicarme qué columnas exactas intervienen para calcular ese margen, y qué tabla concreta es el primer origen de esos valores?

Respuesta:

Esta columna de fact_menu_Sales en gold utiliza coste, precio y ventas de la tabla mi_restaurante.layer30_silver.menu_all_enriched

---

**3. Cambio de fuente entre capas**

Un compañero dice que en Bronze los nombres de restaurantes en ``dim_restaurant`` no están limpios, pero que en Silver sí. 
- ¿Puedes comprobar mediante linaje desde qué tabla se produce realmente la transformación / normalización del nombre del restaurante obteniendo ``restaurante_name_std``?

Respuesta:

esa columna aparece en plata y nace de la columna de bronce ``restaurante`` en bronce

---

**4. Crear una función de Row-Level Security (RLS)**

Contexto de negocio:
Los analistas están asignados por restaurante. Por razones de seguridad y separación de responsabilidades, un analista solo debería ver los datos del restaurante al que pertenece.

juan.poblenou@empresa.com → solo puede ver Restaurante Poblenou

maria.sants@empresa.com → solo puede ver Restaurante Sants

Cualquier otro usuario → no debería ver ninguna fila

Crea y almacena la funcion ``rls_restaurants`` en el esquema ``mi_restaurante.layer99_ops``

In [0]:
%sql
CREATE OR REPLACE FUNCTION mi_restaurante.layer99_ops.rls_restaurants(restaurante STRING)
RETURNS BOOLEAN
RETURN CASE
  WHEN CURRENT_USER() = 'juan.poblenou@empresa.com' AND restaurante = 'Restaurante Poblenou' THEN TRUE
  WHEN CURRENT_USER() = 'maria.sants@empresa.com'  AND restaurante = 'Restaurante Sants' THEN TRUE
  ELSE FALSE
END;


---

**5. Enmascarar / Encriptar valores sensibles**


Contexto real:
El negocio ha detectado que ciertas personas no deberían ver el nombre real del restaurante, por motivos legales o de anonimización de datos para análisis externo.

Se te pide enmascarar el nombre del restaurante cuando el usuario NO es un analista autorizado.

Ejemplo:

juan.analyst@empresa.com → sí puede ver el nombre real

Cualquier otro usuario → verá algo como "**** (oculto)"

In [0]:
%sql
CREATE OR REPLACE FUNCTION mi_restaurante.layer99_ops.fn_mask_restaurante(r STRING)
RETURNS STRING
RETURN
  CASE
    WHEN CURRENT_USER() IN ('juan.analyst@empresa.com', 'maria.analyst@empresa.com')
      THEN r  -- usuario autorizado → ve valor real
    ELSE '*** OCULTO ***'  -- usuario NO autorizado → enmascarado / anonimizado
  END;


In [0]:
%sql
CREATE OR REPLACE VIEW mi_restaurante.layer40_gold.fact_menu_sales_masked AS
SELECT
  fn_mask_restaurante(restaurante) AS restaurante_masked,
  ciudad,
  tipo,
  mes,
  plato_id,
  plato_descripcion,
  ventas_total,
  ingresos,
  margen,
  margen_pct
FROM mi_restaurante.layer40_gold.fact_menu_sales;


Explicación rápida

- No bloquea la fila.
- La fila sigue visible, pero el dato sensible está enmascarado.
- Es diferente a RLS (Row Level Security), donde la fila directamente no se muestra.
- Muy habitual para GDPR, data sharing, o sandbox analítico para externos.